In [1]:
from theano.sandbox import cuda

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 980 (CNMeM is enabled with initial size: 60.0% of memory, cuDNN 5110)


Using TensorFlow backend.


In [2]:
%matplotlib inline
from __future__ import division, print_function

In [3]:
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM#, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam, Nadam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer

from numpy.random import choice

import numpy as np

Using Theano backend.


In [4]:
text = open('successful.txt').read().lower()
print('corpus length:', len(text))

corpus length: 467784


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 72


In [6]:
chars.insert(0, "\0")

In [7]:
''.join(chars[1:-6])

'\n !"#$%&\'()*+,-./0123456789:;=?@[\\]^_`abcdefghijklmnopqrstuvwxyz|'

In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [9]:
idx = [char_indices[c] for c in text]

In [24]:
maxlen = 25
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 467760


In [25]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [26]:
sentences.shape, next_chars.shape

((467758, 25), (467758, 25))

In [27]:
n_fac = 24

In [28]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])

In [29]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [30]:
def print_example():
    seed_string="please give me some pizza"
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-25:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [32]:
%%time
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose=2)

Epoch 1/1
897s - loss: 1.6187
Wall time: 15min 23s


In [33]:
print_example()

please give me some pizza fur rather for the day and some time i knows and they are back to completely helps me distance from my paycheck and at the youte. i our live of broke with a couple cheesy completely running more whulp after the kids ago when i get cash to try that.  i have some come each other redoing.!.hello redditor as a little odge


In [34]:
%%time
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose=2)

Epoch 1/1
860s - loss: 1.3161
Wall time: 14min 20s


In [35]:
print_example()

please give me some pizza and work in such as one of my drawing money and have fawing our partner. anybody the presents of people are being a try to preciate it.  i octupisat (which is my request here. if you can eat anything dinner to hospital. no back on me and could do this, and while he fall off meat pizza would pay it forward once i want 


In [36]:
%%time
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose=2)

Epoch 1/1
855s - loss: 1.2617
Wall time: 14min 15s


In [37]:
print_example()

please give me some pizza in the past two of my money from mittory have run out of food in my car (had to do.

and he's like this. 

i've been so 10 dollars) but i'm broke and public here to read. i am just trying to help me out. i need for rent and i have received been by much expenses... and i'm sure how a serious pizza.  aed father and said


In [38]:
%%time
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose=2)

Epoch 1/1
857s - loss: 1.2328
Wall time: 14min 17s


In [39]:
print_example()

please give me some pizza between rent. it's not sure he is for anything; she been completely use a pizza to find out to rent and loss of college student room at the boys (a'd two would love to use long time. she is trying to thought that worked due to rent my boyfriend of my apartment isn't really awesome! we're first time me (10 delivers now


In [40]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose=2)

Epoch 1/1
858s - loss: 1.2139


In [41]:
print_example()

please give me some pizza if anyone a couple months like everyone! i'm willing to think they rent which is in my father could help us out, reason of the over to be able to help sustain in my account. i got laid off.

i'm in brained, but i've been unemployed back this week when i get paid on thursday. just broke until thursday and i pizza hut, 


In [43]:
model.save_weights('char_rnn.h5')

In [46]:
model.optimizer.lr=0.0001

In [47]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose=2)

Epoch 1/1
886s - loss: 1.1995


In [48]:
print_example()

please give me some pizza and he can spare her friends tonight put, then, i'll have some years ago could really use a pizza tonight, and will absolutely pay it forward.  i could really like to have you a pizza to start. but i just started cy, so it's been making result of course, it will have much easy, i'd give this before pretty much 8 bucks


In [49]:
model.optimizer.lr=0.00001

In [50]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1, verbose=2)

Epoch 1/1
931s - loss: 1.1883


In [53]:
print_example()

please give me some pizza tonight and tomorrow. i have no food in my fridge. my octor maybe too our sick at one of the month, and all of kicket of a second job because of his cupon california, a pizza when we are still no food and buy gift card. so, it's all i am almost druaged and never haven't seen so bany nothing to eat before i live in chi


In [57]:
print_example()

please give me some pizza, it would make our food budget later this guy, but today i'm not the school people delivers or send me everything i would be honest just super food around my mother's only had to buy the loves on any help in a couple of days until then for the past few days before monday. hey guys, it's been my refrigerator enjoying t
